<center><h2>ALTEGRAD Project </h2>
<h3>PREPROCESSING</h3>

<hr>
<span style="font-variant: small-caps;">Xavier Jiménez, Jean Quentin, Sacha Revol</span><br>
<hr>
</center>

# Imports

In [3]:
import networkx as nx
import os
import csv
import numpy as np
import pandas as pd
from random import randint
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss, make_scorer
# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import string
from sklearn.model_selection import cross_val_score
from tqdm import tqdm

# !pip install pip install karateclub
from gensim.models.doc2vec import Doc2Vec
from os import path
import pickle
from scipy import spatial
import random
from sklearn import preprocessing

import re


# Preprocessing

Citation Graph is loaded from `edgelist.txt`

In [3]:
# Create a graph
G = nx.read_edgelist('data/edgelist.txt', delimiter=',', create_using=nx.Graph(), nodetype=int)
nodes = list(G.nodes())
n = G.number_of_nodes()
m = G.number_of_edges()
print('Number of nodes:', n)
print('Number of edges:', m)

Number of nodes: 138499
Number of edges: 1091955


Preprocess `authors.txt` and save it as a dictionary `authors_preprocessed.pkl`

In [28]:
print('Preprocessing authors')
authors = dict()
punctuation = "!#$%&'()*+-./:<=>?@[\]^_`{|}~"
with open('data/authors.txt', 'r', encoding="utf-8") as f:
    for line in tqdm(f):
        node, author = line.split('|--|')
        author = author.lower()
        author = "".join([char for char in author if char not in punctuation])
        author = author.split(',')        
        author[-1] = author[-1].strip()
        authors[int(node)] = author
    a_file = open("data/authors_preprocessed.pkl", "wb")
    pickle.dump(authors, a_file)
    a_file.close()
print('Preprocessing Done')
authors[0]

20330it [00:00, 203034.92it/s]

Preprocessing authors


138499it [00:01, 130319.48it/s]


Preprocessing Done


['james h niblock', 'jianxun peng', 'karen r mcmenemy', 'george w irwin']

In [52]:
# Read the abstract of each paper
try:
    print('Loading Authors preprocessed')
    a_file = open("data/authors_preprocessed.pkl", "rb")
    authors = pickle.load(a_file)
    a_file.close()
except:
    raise SyntaxError("File 'authors_preprocessed.pkl' was not found in 'data/'")

Loading Authors preprocessed


Create a dictionnary from `authors_preprocessed.pkl` with unique names (e.j. "X. Jimenez" and "Xavier Jimenez" are considered as the same author) and saved it as `unique_authors.pkl`.

In [51]:
from namematcher import NameMatcher
from tqdm import tqdm 

def name_similarity(name1, name2):
    """Computes similarity between two names. If similarity is above 0.9 and both names have the
    same letter for the first name, it will return True if one of the authors first name has only
    one letter. False otherwise. If score is 1, returns True.

    Args:
        name1 (str): author name
        name2 (str): author name
        debug (bool, optional): If True, will print authors name and similarity score. Defaults to False.

    Returns:
        bool: True if names are the same. False otherwise.
    """
    if name1 == name2:
        return True
    elif name1.split(" ")[-1] != name2.split(" ")[-1]:
        return False
    
    name_matcher = NameMatcher()
    score = name_matcher.match_names(name1, name2)

    if score >= 0.9:
        if name1.split(" ")[0][0] == name2.split(" ")[0][0]:
            if len(name1.split(" ")[0]) == 1 and len(name2.split(" ")[0]) > 1:
                return True
            elif len(name2.split(" ")[0]) == 1 and len(name1.split(" ")[0]) > 1:
                return True
            else:
                return False
        else:
            return False
    else:
        return False

In [61]:
authors_list = list()
for i in tqdm(range(len(authors))):
    for author in authors[i]:
        authors_list.append(author)
authors_list = list(set(authors_list))

100%|██████████| 138499/138499 [00:00<00:00, 1140952.64it/s]


In [75]:
authors_dict = dict()
for i in tqdm(range(0, 200)):#len(authors_list))):
    similar_authors = list()
    for j in range(0, len(authors_list)):
        if name_similarity(authors_list[i], authors_list[j]):
            similar_authors.append(authors_list[j])
        elif i == j:
            similar_authors.append(authors_list[i])
    authors_dict[authors_list[i]] = np.sort(similar_authors)
    
a_file = open("data/similar_authors_dict.pkl", "wb")
pickle.dump(authors_dict, a_file)
a_file.close()

100%|██████████| 200/200 [00:26<00:00,  7.54it/s]


In [ ]:
try:
    print('Loading similar authors')
    a_file = open("data/similar_authors_dict.pkl", "rb")
    authors_dict = pickle.load(a_file)
    a_file.close()
except:
    raise SyntaxError("File 'similar_authors_dict.pkl' was not found in 'data/'")

unique_authors = dict()
for i in tqdm(range(len(authors))):
    authors_list = list()
    for author in authors[i]:
        authors_list.append(authors_dict[author][0])
unique_authors[i] = authors_list

a_file = open("data/unique_authors_dict.pkl", "wb")
pickle.dump(unique_authors, a_file)
a_file.close()
        

Preprocess `abstracts.txt` and save it as a dictionary `abstract_preprocessed.pkl`. Common NLP operations are made: lowercase, remove punctuation, tokenize, remove stop words.

In [ ]:
print('Preprocessing abstracts')
stop_words = stopwords.words('english')
porter = PorterStemmer()
abstracts = dict()
with open('data/abstracts.txt', 'r', encoding="utf-8") as f:
    for line in tqdm(f):
        node, abstract = line.split('|--|')
        abstract = abstract.lower()
        abstract = "".join([char for char in abstract if char not in string.punctuation])
        abstract = word_tokenize(abstract)
        abstract = [word for word in abstract if word not in stop_words]
        # abstract = [porter.stem(word) for word in abstract]
        abstracts[int(node)] = abstract
a_file = open("data/abstract_preprocessed.pkl", "wb")
pickle.dump(abstracts, a_file)
a_file.close()
print('Preprocessing Done')
abstracts[0][:10]

Citation Graph is loaded from `edgelist.txt` and random edges are removed from it (p=5%). These removed edges will be used to create a validation set that mirrors the test set `test.txt`. New graph is saved as `edgelist_val.txt` and will be loaded as H instead of G. If p is changed, the value should be changed as well on other .ipynb files.

In [ ]:
p = 0.05
    
H = nx.read_edgelist('data/edgelist.txt', delimiter=',', create_using=nx.Graph(), nodetype=int)

with open('data/edgelist.txt') as file:
    lines = file.readlines()

random.seed(42)
indices_to_delete = random.sample(range(len(lines)), int(p * len(lines)))

# sort to delete biggest index first 
indices_to_delete.sort(reverse=True)

for i in tqdm(indices_to_delete):
    line = lines[i]
    t = line.split(',')
    H.remove_edge(int(t[0]), int(t[1]))

nx.readwrite.edgelist.write_edgelist(H, 'data/edgelist_val.txt', delimiter=',')